# Load in all the `.nc` files and merge to one ds

In [2]:
import xarray as xr
import dask 
import matplotlib.pyplot as plt
import numpy as np

### longitude: [-100, -72], latitude: [22, 31]

In [ ]:
ds.nav_lat[-390].values

In [ ]:
ds.nav_lat[-432].values

In [ ]:
ds.nav_lat[-432:-390].values

In [ ]:
ds.nav_lon[:, 1400].values

In [ ]:
ds.nav_lon.values

In [ ]:
ds.votemper.isel(time_counter=0, deptht=0).plot()
plt.grid()

In [ ]:
ds_renamed['longitude'] = np.arange(-180, 180, 1/(1442/360))

In [ ]:
ds_renamed

In [ ]:
ds_newcoord = ds_renamed.assign_coords({'longitude':np.arange(0, 360, 1/(1442/360))})

In [ ]:
ds_newcoord

In [ ]:
ds_renamed.votemper.isel(time=0, depth=0).plot()
plt.grid();

In [ ]:
ds_newcoord.votemper.isel(time=0, depth=0).plot()
plt.grid();

In [ ]:
1442/360

In [ ]:
ds_renamed.coords['longitude'] = (ds_renamed.coords['longitude'] + 180) % 360 - 180

In [ ]:
ds_renamed.longitude

In [ ]:
ds_newcoord.coords['longitude'] = (ds_newcoord.coords['longitude'] + 180) % 360 - 180

In [ ]:
ds_sortby = ds_newcoord.sortby(ds_newcoord.longitude)

In [ ]:
ds_sortby.votemper.isel(time=0, depth=0).plot()
plt.grid();

In [ ]:
np.arange(-180, 180, 1/(4.0055555555555555)).shape

In [ ]:
np.arange(-90, 91, 1/(1021/181)).shape

In [ ]:
ds = ds_renamed.assign_coords({'latitude':np.arange(-90, 91, 1/(1021/181))})
                         # 'longitude':np.arange(-180, 180, 1/(4.0055555555555555))})

In [ ]:
ds

In [ ]:
ds.votemper[0,0].plot()
plt.grid()
plt.hlines(ds.latitude[510],
          xmin=ds.longitude[0], xmax=ds.longitude[-1]);

In [3]:
with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    p2023 = xr.open_dataset('precip-2023.nc', chunks={})

In [3]:
p2023_new = xr.concat([p2023.sel(time=slice(None, '2023-09-01')).isel(expver=0),
                       p2023.sel(time=slice('2023-09-02', None)).isel(expver=1)], dim='time')

In [4]:
p2023_new

<xarray.Dataset>
Dimensions:    (longitude: 1440, latitude: 721, time: 7578)
Coordinates:
  * longitude  (longitude) float32 0.0 0.25 0.5 0.75 ... 359.0 359.2 359.5 359.8
  * latitude   (latitude) float32 90.0 89.75 89.5 89.25 ... -89.5 -89.75 -90.0
    expver     (time) int32 1 1 1 1 1 1 1 1 1 1 1 1 ... 5 5 5 5 5 5 5 5 5 5 5 5
  * time       (time) datetime64[ns] 2023-01-01 ... 2023-11-12T17:00:00
Data variables:
    tp         (time, latitude, longitude) float32 dask.array<chunksize=(5856, 721, 1440), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.6
    history:      2023-11-17 17:33:32 GMT by grib_to_netcdf-2.25.1: /opt/ecmw...

In [6]:
with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    p2023_new.coords['longitude'] = (p2023_new.coords['longitude'] + 180) % 360 - 180
    p2023 = p2023_new.sortby(p2023_new.longitude).sortby(p2023_new.latitude).drop('expver')

In [7]:
p2023.isel(time=slice(None, 3789)).chunk({'time':-1, 'latitude':360, 'longitude':60})

<xarray.Dataset>
Dimensions:    (longitude: 1440, latitude: 721, time: 3789)
Coordinates:
  * longitude  (longitude) float32 -180.0 -179.8 -179.5 ... 179.2 179.5 179.8
  * latitude   (latitude) float32 -90.0 -89.75 -89.5 -89.25 ... 89.5 89.75 90.0
  * time       (time) datetime64[ns] 2023-01-01 ... 2023-06-07T20:00:00
Data variables:
    tp         (time, latitude, longitude) float32 dask.array<chunksize=(3789, 360, 60), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.6
    history:      2023-11-17 17:33:32 GMT by grib_to_netcdf-2.25.1: /opt/ecmw...

In [8]:
p2023.isel(time=slice(3789, None)).chunk({'time':-1, 'latitude':360, 'longitude':60})

<xarray.Dataset>
Dimensions:    (longitude: 1440, latitude: 721, time: 3789)
Coordinates:
  * longitude  (longitude) float32 -180.0 -179.8 -179.5 ... 179.2 179.5 179.8
  * latitude   (latitude) float32 -90.0 -89.75 -89.5 -89.25 ... 89.5 89.75 90.0
  * time       (time) datetime64[ns] 2023-06-07T21:00:00 ... 2023-11-12T17:00:00
Data variables:
    tp         (time, latitude, longitude) float32 dask.array<chunksize=(3789, 360, 60), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.6
    history:      2023-11-17 17:33:32 GMT by grib_to_netcdf-2.25.1: /opt/ecmw...

In [7]:
p2023.sel(time=slice(None, '2023-09-01')).isel(expver=0).drop('expver').chunk({'time':-1, 'latitude':360, 'longitude':60})

<xarray.Dataset>
Dimensions:    (longitude: 1440, latitude: 721, time: 5856)
Coordinates:
  * longitude  (longitude) float32 0.0 0.25 0.5 0.75 ... 359.0 359.2 359.5 359.8
  * latitude   (latitude) float32 90.0 89.75 89.5 89.25 ... -89.5 -89.75 -90.0
  * time       (time) datetime64[ns] 2023-01-01 ... 2023-09-01T23:00:00
Data variables:
    tp         (time, latitude, longitude) float32 dask.array<chunksize=(5856, 360, 60), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.6
    history:      2023-11-17 17:33:32 GMT by grib_to_netcdf-2.25.1: /opt/ecmw...

In [10]:
p2023.sel(time=slice('2023-09-02', None)).isel(expver=1).drop('expver').chunk({'time':-1, 'latitude':360, 'longitude':60})

<xarray.Dataset>
Dimensions:    (longitude: 1440, latitude: 721, time: 1722)
Coordinates:
  * longitude  (longitude) float32 0.0 0.25 0.5 0.75 ... 359.0 359.2 359.5 359.8
  * latitude   (latitude) float32 90.0 89.75 89.5 89.25 ... -89.5 -89.75 -90.0
  * time       (time) datetime64[ns] 2023-09-02 ... 2023-11-12T17:00:00
Data variables:
    tp         (time, latitude, longitude) float32 dask.array<chunksize=(1722, 360, 60), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.6
    history:      2023-11-17 17:33:32 GMT by grib_to_netcdf-2.25.1: /opt/ecmw...

In [11]:
#uploading global surf_ds
import json
import gcsfs

with open('/home/jovyan/pangeo-181919-f141378dc975.json') as token_file:
    token = json.load(token_file)
gcfs_w_token = gcsfs.GCSFileSystem(token=token)


gcsmap_ds = gcfs_w_token.get_mapper('pangeo-tmp/stb2145/ERA5/datasets/precip-2023-expver1.zarr')
p2023.sel(time=slice('2023-09-02', None)).isel(expver=1).drop('expver').chunk({'time':-1, 'latitude':360, 'longitude':60}).to_zarr(gcsmap_ds, mode='w')

In [16]:
with open('/home/jovyan/pangeo-181919-f141378dc975.json') as token_file:
    token = json.load(token_file)
gcfs_w_token = gcsfs.GCSFileSystem(token=token)

def open_ds(year):
    with dask.config.set(**{'array.slicing.split_large_chunks': True}):
        ds = xr.open_zarr(gcfs_w_token.get_mapper(f'pangeo-tmp/stb2145/ERA5/datasets/precip_{year}.zarr'))
    return ds

with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    p23_0 = xr.open_zarr(gcfs_w_token.get_mapper(f'pangeo-tmp/stb2145/ERA5/datasets/precip-2023-expver0.zarr'))
    p23_1 = xr.open_zarr(gcfs_w_token.get_mapper(f'pangeo-tmp/stb2145/ERA5/datasets/precip-2023-expver1.zarr'))

In [17]:
with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    p23_unsorted = xr.concat([p23_0, p23_1], dim='time')
    p23_unsorted.coords['longitude'] = (p23_unsorted.coords['longitude'] + 180) % 360 - 180
    p23 = p23_unsorted.sortby(p23_unsorted.longitude).sortby(p23_unsorted.latitude)

In [18]:
p18 = open_ds(18)
p19 = open_ds(19)
p20 = open_ds(20)
p21 = open_ds(20)
p22 = open_ds(20)

In [19]:
pds = xr.concat([p18, p19, p20, p21, p22, p23], dim='time')

In [31]:
51450/8760

5.873287671232877

In [30]:
pds.chunk({'time':8760, 'latitude':360, 'longitude':60})

<xarray.Dataset>
Dimensions:    (latitude: 721, longitude: 1440, time: 51450)
Coordinates:
  * latitude   (latitude) float32 -90.0 -89.75 -89.5 -89.25 ... 89.5 89.75 90.0
  * longitude  (longitude) float32 -180.0 -179.8 -179.5 ... 179.2 179.5 179.8
  * time       (time) datetime64[ns] 2018-01-01 ... 2023-11-12T17:00:00
Data variables:
    tp         (time, latitude, longitude) float32 dask.array<chunksize=(8760, 360, 60), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.6
    history:      2023-11-07 18:38:21 GMT by grib_to_netcdf-2.25.1: /opt/ecmw...

In [22]:
from dask_gateway import GatewayCluster

cluster = GatewayCluster()
cluster.scale(30)
#cluster.adapt(minimum=2, maximum=10)  # or cluster.scale(n) to a fixed size.
client = cluster.get_client()

In [23]:
client

Connection method: Cluster object,Cluster type: dask_gateway.GatewayCluster
Dashboard: /services/dask-gateway/clusters/prod.a2045c50b67c4c93be13c1211cff1b1c/status,


In [33]:
#uploading global surf_ds
import json
import gcsfs

with open('/home/jovyan/pangeo-181919-f141378dc975.json') as token_file:
    token = json.load(token_file)
gcfs_w_token = gcsfs.GCSFileSystem(token=token)


gcsmap_ds = gcfs_w_token.get_mapper('pangeo-tmp/stb2145/ERA5/datasets/precip-2018_2023.zarr')
pds.chunk({'time':8760, 'latitude':361, 'longitude':6}).to_zarr(gcsmap_ds, mode='w')

In [34]:
check = xr.open_zarr('pangeo-tmp/stb2145/ERA5/datasets/precip-2018_2023.zarr')

FileNotFoundError: No such file or directory: '/home/jovyan/MHW-WMT/pangeo-tmp/stb2145/ERA5/datasets/precip-2018_2023.zarr'